<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/banner_IA.png"  width="1000px" height="200px">

# **Taller 06:  Regression**

## **Outline**

1. [Ejercicio 1.](#eje1)
2. [Ejercicio 2.](#eje2)
3. [Ejercicio 3.](#eje3)
4. [Ejercicio 4.](#eje4)
5. [Ejercicio 5.](#eje5)



In [5]:
#@title **Execute this cell**
#@markdown Please include your student id
import sys
import inspect

group_id = "IA1-20252-G1" #@param {type:"string"}
assignment_id = group_id +'.taller_regression'
student_id = "2225112" #@param {type:"string"}
"""
Put your student ID here

Example: student_id =  '2152145'
"""

"\nPut your student ID here\n\nExample: student_id =  '2152145'\n"

In [6]:
 #@title **Execute this cell**
#@markdown **UTILS**
#@markdown Please dont modify any line in this cell

import os
import json
import requests
from collections import namedtuple


Config = namedtuple('Config', ['server_name'])
config = Config(server_name='https://bivlabgrader.azurewebsites.net/api')


def check_solution_and_evaluate(assignment_id: str, student_func_str: str):

    # Set the endpoint and payload.
    payload = {
        'func_str': student_func_str,
        'assignment_id': assignment_id,
        'student_id': student_id
    }
    endpoint_url = config.server_name + '/CheckAndEvaluateSolution'
    # print(endpoint_url)

    # Make request to server with the data coming from the notebook.
    r = requests.post(endpoint_url, params=payload)
    pprint_json_response(r.json())
    return r


def pprint_json_response(response, indent=0):
    """Pretty print the response."""
    for key, value in response.items():
        print('\t' * indent + str(key.capitalize()))

        # If dictionary, do a recurrent call.
        if isinstance(value, dict):
            pprint_json_response(value, indent + 1)
        else:
            # Enumerate elements if list.
            if isinstance(value, list):
                if len(value) == 1:
                    print('\t' * (indent + 1) + str(value[0]))
                else:
                    for i, e in enumerate(value, start=1):
                        print('\t' * (indent + 1) + f'{i}. {e}')
            else:
                print('\t' * (indent + 1) + str(value))

In [7]:
#@title **Import libraries**

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(21)

import warnings
warnings.filterwarnings('ignore')

---
# **Ejercicio 1**  <a name="eje1"></a>
---

# Regression

## Contexto
El dataset "Dogs of Zurich" (https://www.kaggle.com/datasets/kmader/dogs-of-zurich) contiene información sobre datos de dueños y mascotas en la ciudad de Zurich.

In [4]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1z0DCJFIFyLIObEBjhsm66RoVwRq7YYoj", sep=";")
df.head()

,HALTER_ID,ALTER,GESCHLECHT,STADTKREIS,STADTQUARTIER,RASSE1,RASSENTYP,GEBURTSJAHR_HUND,GESCHLECHT_HUND,HUNDEFARBE
0,126,51-60,m,9.0,92.0,Welsh Terrier,K,2011,w,schwarz/braun
1,171,61-70,m,3.0,31.0,Berner Sennenhund,I,2009,m,tricolor
2,574,61-70,w,2.0,23.0,Cairn Terrier,K,2002,w,brindle
3,695,41-50,m,6.0,63.0,Labrador Retriever,I,2012,w,braun
4,893,61-70,w,7.0,71.0,Mittelschnauzer,I,2010,w,schwarz


## Tu tarea
Desarrolle una función que **reciba** un dataset (`df`) y que:
- Elimine los valores nulos del dataset.
- Elimine a las columnas `["HALTER_ID", "ALTER", "STADTKREIS", "STADTQUARTIER"]`
- Reemplace los valores "m" por cero (0), y "w" por uno (1).
- Convierta las columnas categóricas `["RASSE1"], ["RASSENTYP"], ["HUNDEFARBE"]` a numéricas.
- Considere como ground truth a la columna ["GEBURTSJAHR_HUND"] (el año de nacimiento de la mascota).
- Destine un 85% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`
- Entrene un estimador `RandomForestRegressor` con parámetros `n_estimators=100` y `random_state=21`
- **Devuelva** el `mean_squared_error`
- **Devuelva** el `mean_absolute_error`

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [5]:
#@title **code student**

def taller06_20252_p01(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    np.random.seed(21)

    df = df.dropna()
    df = df.drop(columns=["HALTER_ID", "ALTER", "STADTKREIS", "STADTQUARTIER"], axis=1)
    
    df["GESCHLECHT"] = df["GESCHLECHT"].replace({"m":0, "w":1})
    df["GESCHLECHT_HUND"] = df["GESCHLECHT_HUND"].replace({"m":0, "w":1})
    
    df["RASSE1"] = df["RASSE1"].factorize()[0]
    df["RASSENTYP"] = df["RASSENTYP"].factorize()[0]
    df["HUNDEFARBE"] = df["HUNDEFARBE"].factorize()[0]
    
    y = df["GEBURTSJAHR_HUND"]
    x = df.drop(columns=["GEBURTSJAHR_HUND"], axis=1)
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=21)
    
    regressor = RandomForestRegressor(random_state=21, n_estimators=100)
    regressor.fit(x_train, y_train)


    return mean_squared_error(y_test, regressor.predict(x_test)), mean_absolute_error(y_test, regressor.predict(x_test))

In [6]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1z0DCJFIFyLIObEBjhsm66RoVwRq7YYoj", sep=";")
taller06_20252_p01(df)

(182.48086328075894, 3.835977947525619)

## ✅ Salidas esperadas

(182.48086328075894, 3.835977947525619)


In [7]:
#@title **send your answer**
student_func_str = inspect.getsource(taller06_20252_p01)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, both regression metrics are correct.


---
# **Ejercicio 2**  <a name="eje2"></a>
---

# Regresión

## Contexto
El dataset "80 Cereals" (https://www.kaggle.com/datasets/crawford/80-cereals) contiene información sobre distintas marcas de cereales, sus fabricantes, información nutricional, entre otros.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


## Tu Tarea
Desarrolle una función que **reciba** un dataset (`df`) y que:
- Elimine los valores nulos del dataset.
- Elimine la columna `["name"]`
- Convierta las columnas categóricas `["mfr"]`, `["type"]` a numéricas.
- Considere como ground truth a la columna `["rating"]` (puntaje del cereal).
- Destine un 90% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`
- Entrene un estimador `DecisionTreeRegressor` con parámetros `max_depth=100` y `random_state=21`
- **Devuelva** el `mean_absolute_error` para este estimador.
- **Devuelva** el `mean_squared_error` para este estimador.

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [8]:
#@title **code student**

def taller06_20252_p02(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    np.random.seed(21)

    df = df.dropna()
    df = df.drop(columns=["name"], axis=1)
    
    df["mfr"] = df["mfr"].factorize()[0]
    df["type"] = df["type"].factorize()[0]
    
    y = df["rating"]
    x = df.drop(columns=["rating"], axis=1)
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=21)
    regressor = DecisionTreeRegressor(random_state=21, max_depth=100)
    regressor.fit(x_train, y_train)

    return mean_absolute_error(y_test, regressor.predict(x_test)), mean_squared_error(y_test, regressor.predict(x_test))

In [9]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
taller06_20252_p02(df)

(8.718688125, 183.17935098008306)

## ✅ Salidas esperadas

(8.718688125, 183.17935098008306)


In [10]:
#@title **send your answer**
student_func_str = inspect.getsource(taller06_20252_p02)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, both regression metrics are correct.


---
# **Ejercicio 3**  <a name="eje3"></a>
---

## Regression: Feature importances

## Contexto
El dataset `supermarket sales` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/supermarket_sales.csv) contiene información relacionada con productos, costos, ganancias y otros aspectos de un supermercado.

In [1]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Tu tarea

Implementar una función que **reciba** como parámetro el dataset (`df`), y que:
- Elimine las columnas `["Invoice ID", "Date", "Time"]`
- Convierta las columnas categóricas `["Branch"], ["City"], ["Customer type"], ["Gender"], ["Product line"], ["Payment"]` a numéricas.
- El ground truth será la columna `["Total"]`
- Utilice un 90% del dataset para entrenamiento, con parámetros `test_size=0.1, random_state=21`)
- Entrene un estimador `DecisionTreeRegressor` con parámetros `random_state=21`
- **Devuelva** el `mean_absolute_error` para este estimador.
- Calcule las columnas más importantes del dataset.

Su nuevo dataset será: las 3 (tres) columnas <ins>**MÁS IMPORTANTES**</ins>.<br>Con esta nueva configuración:
- Entrene un estimador `SVR` con parámetros `kernel='linear'`
- **Devuelva** el `mean_absolute_error` para este estimador.






<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [ ]:
#@title **code student**

def taller06_20252_p03(df):
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.svm import SVR
  from sklearn.metrics import mean_absolute_error, mean_squared_error
  np.random.seed(21)
  
  df = df.drop(columns=["Invoice ID", "Date", "Time"], axis=1)
  
  df["Branch"] = df["Branch"].factorize()[0]
  df["City"] = df["City"].factorize()[0]
  df["Customer type"] = df["Customer type"].factorize()[0]
  df["Gender"] = df["Gender"].factorize()[0]
  df["Product line"] = df["Product line"].factorize()[0]
  df["Payment"] = df["Payment"].factorize()[0]
  
  y = df["Total"]
  x = df.drop(columns=["Total"], axis=1)
  
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=21)
  regressor = DecisionTreeRegressor(random_state=21)
  regressor.fit(x_train, y_train)
  abs_tree = mean_absolute_error(y_test, regressor.predict(x_test))
  
  top_3 = pd.Series(regressor.feature_importances_, index=x.columns).sort_values(ascending=False).index[:3].tolist()
  
  df2 = df[top_3 + ["Total"]]
  y2 = df2["Total"]
  x2 = df2.drop(columns=["Total"], axis=1)
  
  x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.1, random_state=21)
  regressor_svr = SVR(kernel="linear")
  regressor_svr.fit(x2_train, y2_train)
  abs_svr = mean_absolute_error(y2_test, regressor_svr.predict(x2_test))

  return abs_tree, abs_svr

In [3]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
taller06_20252_p03(df)

(1.1017649999999992, 0.054950240193541756)

## ✅ Salidas esperadas

(1.1017649999999992, 0.054950240193541756)


In [8]:
#@title **send your answer**
student_func_str = inspect.getsource(taller06_20252_p03)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, both regression metrics are correct.


---
# **Ejercicio 4**  <a name="eje4"></a>
---

## Regression: Cross Validation

## Contexto
El dataset `supermarket sales` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/supermarket_sales.csv) contiene información relacionada con productos, costos, ganancias y otros aspectos de un supermercado.

In [9]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Tu tarea

Implementar una función que **reciba** como parámetro el dataset (`df`), y que:
- Elimine las columnas `["Invoice ID", "Date", "Time", "gross margin percentage"]`
- Convierta las columnas categóricas `["Branch"], ["City"], ["Customer type"], ["Gender"], ["Product line"], ["Payment"]` a numéricas.
- El ground truth será la columna `["Unit price"]`
- Utilice un 75% del dataset para entrenamiento, con parámetros `test_size=0.25, random_state=21`)
- Entrene un estimador SVR con parámetros `kernel="poly"
- **Devuelva** el `mean_absolute_error` del estimador
- Entrene el mismo estimador, mediante `cross validation` con parámetros `cv=KFold(3, shuffle=False), scoring=make_scorer(mean_absolute_error)`
- **Devuelva** el promedio de los resultados obtenidos con este método
- **Devuelva** la desviación de los resultados obtenidos con este método

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [12]:
#@title **code student**

def taller06_20252_p04(df):
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import mean_absolute_error, make_scorer
  from sklearn.svm import SVR
  from sklearn.model_selection import cross_val_score
  from sklearn.model_selection import KFold
  np.random.seed(21)

  df = df.drop(columns=["Invoice ID", "Date", "Time","gross margin percentage"], axis=1)
  df["Branch"] = df["Branch"].factorize()[0]
  df["City"] = df["City"].factorize()[0]
  df["Customer type"] = df["Customer type"].factorize()[0]
  df["Gender"] = df["Gender"].factorize()[0]
  df["Product line"] = df["Product line"].factorize()[0]
  df["Payment"] = df["Payment"].factorize()[0]

  y = df["Unit price"]
  x = df.drop(columns=["Unit price"], axis=1)
  
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=21)
  
  regressor  = SVR(kernel="poly")
  regressor.fit(x_train, y_train)
  
  mae_svr = mean_absolute_error(y_test, regressor.predict(x_test))
  
  s = cross_val_score(regressor, x, y, cv=KFold(3, shuffle=False), scoring=make_scorer(mean_absolute_error))
  mean = s.mean()
  std = s.std()
  

  return mae_svr, mean, std

In [13]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
taller06_20252_p04(df)

(17.96167100300541,
 np.float64(18.627417627666095),
 np.float64(0.25937617241624805))

## ✅ Salidas esperadas

(17.96167100300541,<br>
 np.float64(18.627417627666095),<br>
 np.float64(0.25937617241624805))

In [14]:
#@title **send your answer**
student_func_str = inspect.getsource(taller06_20252_p04)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all regression metrics are correct.


---
# **Ejercicio 5**  <a name="eje5"></a>
---

# Cross validation: leave one out

## Contexto:
Considere el dataset `penguins size` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/penguins_size.csv) el cual contiene datos sobre las dimensiones de una población de pinguinos.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Tu tarea
Implemente una función que **reciba** un dataset (`df`) y que:
- **Elimine** los valores nulos del dataset.
- Convierta las columnas categóricas `["species"], ["island"], ["sex"]` a numéricas.
- Considere a la columna `["flipper_length_mm"]` como el ground truth.
- Lleve a cabo un entrenamiento mediante la variante `leave one out` de cross validation.
- El estimador a ser entrenado debe ser un `SVR` con parámetros `kernel="poly", random_state=21`.<br>
- La métrica a evaluar será `mean_absolute_error`
- **Devuelva** el promedio de los resultados obtenidos
- **Devuelva** la desviación de los resultados obtenidos


<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [17]:
#@title **code student**

def taller06_20252_p05(df):
  import pandas as pd
  import numpy as np
  from sklearn.svm import SVR
  from sklearn.model_selection import LeaveOneOut
  from sklearn.model_selection import cross_val_score
  from sklearn.metrics import mean_absolute_error, make_scorer
  np.random.seed(21)

  df = df.dropna()
  
  df["species"] = df["species"].factorize()[0]
  df["island"] = df["island"].factorize()[0]
  df["sex"] = df["sex"].factorize()[0]

  y = df["flipper_length_mm"]
  x = df.drop(columns=["flipper_length_mm"], axis=1)
  
  regressor = SVR(kernel="poly")
  loo = LeaveOneOut()
  
  s = cross_val_score(regressor, x, y, cv=loo, scoring=make_scorer(mean_absolute_error))
  return s.mean(), s.std()

In [18]:
#@title **check your answer**
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
taller06_20252_p05(df)

(np.float64(5.727572811690977), np.float64(4.172214046757537))

## ✅ Salidas esperadas

(np.float64(5.727572811690977), np.float64(4.172214046757537))


In [19]:
#@title **send your answer**
student_func_str = inspect.getsource(taller06_20252_p05)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, both mean and std of MAE are correct.


---
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/bannerThanks.jpg" alt="Drawing" style="width:700px;"/>